In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress

# Study data files
cobra_09_path = "data/COBRA-2009-2019.csv"
cobra_20_path = "data/COBRA-2020.csv"
cobra_21_path = "data/COBRA-2021.csv"
county_cases_path = "data/county_cases.csv"

# Read the mouse data and the study results
cobra_09 = pd.read_csv(cobra_09_path)
cobra_20 = pd.read_csv(cobra_20_path)
cobra_21 = pd.read_csv(cobra_21_path)
county_cases = pd.read_csv(county_cases_path)


# Combine the data into a single dataset
#study_data_complete = pd.merge(study_results, mouse_metadata, how="left", on="Mouse ID")

# Display the data table for preview
#study_data_complete

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (3,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
cobra_09.columns = ['offense_id','rpt_date','occur_date','occur_time','poss_date','poss_time','beat','apt_office_prefix', 'apt_office_num', 'location', 'shift_occurence','location_type','UC2_Literal','UC2#','ibr_code','neighborhood','npu','lat','long']
cobra_09 = cobra_09.drop(columns = ['apt_office_prefix','apt_office_num','shift_occurence','location_type','UC2#'])
cobra_09['rpt_date']= pd.to_datetime(cobra_09['rpt_date'])
cobra_09['occur_date']= pd.to_datetime(cobra_09['occur_date'])
cobra_09['poss_date']= pd.to_datetime(cobra_09['poss_date'])
cobra_09['rpt_date'] = cobra_09['rpt_date'].dt.strftime("%m/%d/%y")
cobra_09['occur_date'] = cobra_09['occur_date'].dt.strftime("%m/%d/%y")
cobra_09['poss_date'] = cobra_09['poss_date'].dt.strftime("%m/%d/%y")
cobra_09.head()

,offense_id,rpt_date,occur_date,occur_time,poss_date,poss_time,beat,location,UC2_Literal,ibr_code,neighborhood,npu,lat,long
0,90010930,01/01/09,01/01/09,1145,01/01/09,1148.0,411.0,2841 GREENBRIAR PKWY,LARCENY-NON VEHICLE,2303,Greenbriar,R,33.68845,-84.49328
1,90011083,01/01/09,01/01/09,1330,01/01/09,1330.0,511.0,12 BROAD ST SW,LARCENY-NON VEHICLE,2303,Downtown,M,33.75320,-84.39201
2,90011208,01/01/09,01/01/09,1500,01/01/09,1520.0,407.0,3500 MARTIN L KING JR DR SW,LARCENY-NON VEHICLE,2303,Adamsville,H,33.75735,-84.50282
3,90011218,01/01/09,01/01/09,1450,01/01/09,1510.0,210.0,3393 PEACHTREE RD NE,LARCENY-NON VEHICLE,2303,Lenox,B,33.84676,-84.36212
4,90011289,01/01/09,01/01/09,1600,01/01/09,1700.0,411.0,2841 GREENBRIAR PKWY SW,LARCENY-NON VEHICLE,2303,Greenbriar,R,33.68677,-84.49773


In [3]:
cobra_20 = cobra_20.drop(columns = ['apt_office_prefix','apt_office_num','MinOfucr','dispo_code','Shift','loc_type'])
cobra_20_reorder = ['offense_id','rpt_date','occur_date','occur_time','poss_date','poss_time','beat','location','ibr_code','UC2_Literal','neighborhood','npu','lat','long']
cobra_20 = cobra_20.reindex(columns = cobra_20_reorder)
#df = df.reindex(columns=['Name', 'Gender', 'Age', 'City', 'Education'])
cobra_20.head()

,offense_id,rpt_date,occur_date,occur_time,poss_date,poss_time,beat,location,ibr_code,UC2_Literal,neighborhood,npu,lat,long
0,203341346,12/31/2020,11/29/2020,19:21,11/29/2020,23:50,405,3050 M.L.K. JR DR SW,13A,AGG ASSAULT,Harland Terrace,I,33.751060,-84.486163
1,203631200,12/31/2020,12/28/2020,17:21,12/28/2020,17:45,302,1186 IRA ST SW,23H,LARCENY-NON VEHICLE,Pittsburgh,V,33.722696,-84.399763
2,203631285,12/31/2020,12/24/2020,14:00,12/24/2020,18:26,603,501 BOULEVARD PL NE,23H,LARCENY-NON VEHICLE,Old Fourth Ward,M,33.769680,-84.370736
3,203651634,12/31/2020,12/30/2020,20:15,12/30/2020,22:25,211,3101 PIEDMONT RD NE,23F,LARCENY-FROM VEHICLE,Peachtree Park,B,33.840653,-84.369361
4,203660108,12/31/2020,12/31/2020,1:15,12/31/2020,1:15,306,1974 SYLVAN RD SW,13A,AGG ASSAULT,Sylvan Hills,X,33.700432,-84.418128


In [4]:
cobra_21 = cobra_21.drop(columns = ['occur_day','occur_day_num','occur_day_num','zone'])
cobra_21.head()

,offense_id,rpt_date,occur_date,occur_time,poss_date,poss_time,beat,location,ibr_code,UC2_Literal,neighborhood,npu,lat,long
0,21140096,5/20/2021,5/19/2021,15:20,5/19/2021,15:25,103,"1720 MARIETTA BLVD NW\nATLANTA, GA 30318\nUNIT...",23F,LARCENY-FROM VEHICLE,Hills Park,D,33.801693,-84.436453
1,21282015,8/6/2021,8/6/2021,22:23,8/6/2021,22:24,508,"300 PEACHTREE ST NE\nATLANTA, GA 30308\nUNITED...",23F,LARCENY-FROM VEHICLE,Downtown,M,33.762549,-84.387874
2,21791668,6/28/2021,6/27/2021,21:17,6/28/2021,20:00,102,"870 MAYSON TURNER RD NW\nATLANTA, GA 30314\nUN...",23H,LARCENY-NON VEHICLE,Vine City,L,33.755768,-84.416631
3,200041082,1/5/2021,1/4/2021,16:00,1/4/2021,17:00,605,"275 MEMORIAL DR SE\nATLANTA, GEORGIA 30312\nUN...",23C,LARCENY-NON VEHICLE,Grant Park,W,33.746503,-84.378535
4,202611428,9/18/2021,9/18/2021,18:04,9/18/2021,18:15,112,"2750 DONALD LEE HOLLOWELL PKWY NW\nATLANTA, GA...",23C,LARCENY-NON VEHICLE,Collier Heights,I,33.777383,-84.477433


In [6]:
concat_list = [cobra_09, cobra_20, cobra_21]  # List of your dataframes
crime_df_complete = pd.concat(concat_list)
crime_df_complete

,offense_id,rpt_date,occur_date,occur_time,poss_date,poss_time,beat,location,UC2_Literal,ibr_code,neighborhood,npu,lat,long
0,90010930,01/01/09,01/01/09,1145,01/01/09,1148.0,411.0,2841 GREENBRIAR PKWY,LARCENY-NON VEHICLE,2303,Greenbriar,R,33.688450,-84.493280
1,90011083,01/01/09,01/01/09,1330,01/01/09,1330.0,511.0,12 BROAD ST SW,LARCENY-NON VEHICLE,2303,Downtown,M,33.753200,-84.392010
2,90011208,01/01/09,01/01/09,1500,01/01/09,1520.0,407.0,3500 MARTIN L KING JR DR SW,LARCENY-NON VEHICLE,2303,Adamsville,H,33.757350,-84.502820
3,90011218,01/01/09,01/01/09,1450,01/01/09,1510.0,210.0,3393 PEACHTREE RD NE,LARCENY-NON VEHICLE,2303,Lenox,B,33.846760,-84.362120
4,90011289,01/01/09,01/01/09,1600,01/01/09,1700.0,411.0,2841 GREENBRIAR PKWY SW,LARCENY-NON VEHICLE,2303,Greenbriar,R,33.686770,-84.497730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17627,212940064,10/21/2021,10/21/2021,02:24,10/21/2021,02:25,112,"3200 DONALD LEE HOLLOWELL PKWY NW\nATLANTA, GA...",LARCENY-FROM VEHICLE,23F,Collier Heights,I,33.786330,-84.492188
17628,212940073,10/21/2021,10/21/2021,00:35,10/21/2021,00:45,111,"71 LINKWOOD RD NW\nATL, GA 30311\nUNITED STATES",AGG ASSAULT,13A,Westhaven,I,33.757226,-84.482355
17629,212940212,10/21/2021,10/21/2021,02:49,10/21/2021,03:28,409,"3113 CAMPBELLTON RD SW\nATLANTA, GA 30311\nUNI...",AUTO THEFT,240,Southwest,R,33.692468,-84.490825
17630,213000875,8/18/2021,7/10/2021,16:00,8/18/2021,12:14,612,"491 FLAT SHOALS AVE SE\nATLANTA, GA 30316\nUNI...",LARCENY-FROM VEHICLE,23F,East Atlanta,W,33.740555,-84.346336


In [5]:
county_cases.columns = ['County Name','Cases','County ID','State FIPS Code','County FIPS Code','Population','Hospitalization','Deaths', 'Case Rate', 'Death Rate', '14 Day Case Rate','14 Day Cases','Antigen Cases','Probable Deaths']
county_cases.head()

,County Name,Cases,County ID,State FIPS Code,County FIPS Code,Population,Hospitalization,Deaths,Case Rate,Death Rate,14 Day Case Rate,14 Day Cases,Antigen Cases,Probable Deaths
0,Appling,2761,US-13001,13,1,18561,234,94,14875.28,506.44,269.38,50,720,9
1,Atkinson,1171,US-13003,13,3,8330,172,29,14057.62,348.14,252.10,21,395,5
2,Bacon,1856,US-13005,13,5,11404,147,47,16274.99,412.14,306.91,35,646,11
3,Baker,241,US-13007,13,7,3116,43,13,7734.27,417.20,64.18,2,138,1
4,Baldwin,5232,US-13009,13,9,44428,381,162,11776.36,364.63,126.05,56,1399,22
